In [17]:
from CurvyCUSIPs.CurveDataFetcher import CurveDataFetcher 
from datetime import datetime
import pandas as pd
import scipy

# to run async in ipynb
import nest_asyncio
nest_asyncio.apply()

import os
from dotenv import dotenv_values
config = dotenv_values(os.path.join(os.getcwd(), "../.env"))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Fetching with `CurveDataFetcher`:
- Timeseries by CUSIP
- CT EOD, Bid, Offer, Mid Yields sourced from FedInvest
- CMT Yields from FRED (Fed) or Treasury - Both sources likely pull from the same/use same par curve build (Treasury's monotone convex interpolation performed on forward rates from bids around 3:30 PM EST by FRBNY)
- Pull All Active CUSIPs given date

In [2]:
cusip_curve_builder = CurveDataFetcher(use_ust_issue_date=True, fred_api_key=config["FRED_API_KEY"])

# See Historical UST Auctions:

In [3]:
cusip_curve_builder.ust_data_fetcher._historical_auctions_df

,cusip,security_type,auction_date,issue_date,maturity_date,price_per100,allocation_pctage,avg_med_yield,bid_to_cover_ratio,comp_accepted,...,original_security_term,security_term_week_year,primary_dealer_accepted,primary_dealer_tendered,reopening,total_accepted,total_tendered,treas_retail_accepted,treas_retail_tenders_accepted,ust_label
298,912797LB1,Bill,2024-11-12,2024-11-14,2025-05-15,null,null,null,null,null,...,52-Week,26-Week,null,null,Yes,null,null,null,Yes,nan% May-25
297,912797MK0,Bill,2024-11-12,2024-11-14,2025-02-13,null,null,null,null,null,...,26-Week,13-Week,null,null,Yes,null,null,null,Yes,nan% Feb-25
296,912796ZV4,CMB,2024-11-12,2024-11-14,2024-12-26,null,null,null,null,null,...,52-Week,0-Week,null,null,Yes,null,null,null,No,nan% Dec-24
294,912797MN4,Bill,2024-11-07,2024-11-12,2024-12-10,99.648833,20.190000,null,2.630000,88700873600,...,17-Week,4-Week,27225950000,168900000000,Yes,95274707000,249764577000,5160639100,Yes,4.594% Dec-24
293,912797MW4,Bill,2024-11-07,2024-11-12,2025-01-07,99.301556,56.070000,null,2.590000,88254854000,...,17-Week,8-Week,35611885000,160750000000,Yes,90260736900,233761556900,1167797200,Yes,4.584% Jan-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,9127935W2,Bill,1980-01-02,1980-01-08,1981-01-02,null,3.000000,null,null,null,...,52-Week,52-Week,null,null,No,4007825000,6329685000,null,null,12.160% Jan-81
2,9127933Y0,Bill,1979-12-28,1980-01-03,1980-04-03,null,21.000000,null,null,null,...,null,13-Week,null,null,Yes,3257440000,5371540000,null,null,12.590% Apr-80
3,9127934U7,Bill,1979-12-28,1980-01-03,1980-07-03,null,94.000000,null,null,null,...,26-Week,26-Week,null,null,No,3348725000,5954350000,null,null,12.800% Jul-80
1,912810CK2,Bond,1979-11-01,1979-11-15,2009-11-15,null,44.000000,10.440000,null,null,...,30-Year,30-Year,null,null,No,2315000000,3594000000,null,null,10.375% Nov-09


# Fetching Timeseries data by CUSIPs: 
Sources:
- FedInvest
- FedInvest (Hosted on Github)
- Public.com
- Bondsupermart
- Cbond
- Todos:
  - Yahoo finance (via Dusseldorf, Munich, Stuttgart, TLO) (downside all in CEST)
  - directly from the euro exchanges (in CEST)
  - Webull
  - WSJ
  - other brokers?

### Directly from FedInvest - use `CurveDataFetcher.fedinvest_data_fetcher.cusips_timeseries`:

In [4]:
cusip_dict_df = cusip_curve_builder.fedinvest_data_fetcher.cusips_timeseries(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 8, 20),
    cusips=["912810UA4", "912810UC0", "912810ES3"],
)
cusip_dict_df

{'912810UA4':          Date  bid_price  offer_price   mid_price  eod_price  bid_yield  \
 0  2024-05-16  102.00000   102.015625  102.007812  101.71875   4.502775   
 1  2024-05-17  101.31250   101.312500  101.312500  101.03125   4.544326   
 2  2024-05-20  100.68750   100.703125  100.695312  100.78125   4.582495   
 3  2024-05-21  101.15625   101.156250  101.156250  101.18750   4.553803   
 4  2024-05-22  101.25000   101.265625  101.257812  101.28125   4.548073   
 ..        ...        ...          ...         ...        ...        ...   
 61 2024-08-14  107.75000   107.765625  107.757812  107.84375   4.167387   
 62 2024-08-15  108.62500   108.640625  108.632812  108.56250   4.118731   
 63 2024-08-16  107.21875   107.234375  107.226562  107.50000   4.197114   
 64 2024-08-19  109.15625   109.171875  109.164062  109.37500   4.089378   
 65 2024-08-20  107.56250   107.578125  107.570312  107.90625   4.177765   
 
     offer_yield  mid_yield  eod_yield  
 0      4.501834   4.502304   4.

### Directly from Public.com - use `CurveDataFetcher.publicdotcom_data_fetcher.public_dotcom_timeseries_api`:

In [5]:
cusip_dict_df = cusip_curve_builder.publicdotcom_data_fetcher.public_dotcom_timeseries_api(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 8, 20),
    cusips=["912810UA4", "912810UC0", "912810ES3"],
)
cusip_dict_df

{'912810UA4':                         Date     Price       YTM
 0  2024-05-16 13:30:00+00:00  101.6930  4.521000
 1  2024-05-17 13:30:00+00:00  101.0625  4.560000
 2  2024-05-20 13:30:00+00:00  100.7205  4.580500
 3  2024-05-21 13:30:00+00:00  101.2555  4.548000
 4  2024-05-22 13:30:00+00:00  101.3360  4.543000
 ..                       ...       ...       ...
 61 2024-08-14 13:30:00+00:00  108.4344  4.129614
 62 2024-08-15 13:30:00+00:00  107.3515  4.190031
 63 2024-08-16 13:30:00+00:00  107.9047  4.158988
 64 2024-08-19 13:30:00+00:00  108.2903  4.137499
 65 2024-08-20 13:30:00+00:00  109.2759  4.083097
 
 [66 rows x 3 columns],
 '912810UC0':                        Date     Price       YTM
 0 2024-08-16 13:30:00+00:00  101.6405  4.146500
 1 2024-08-19 13:30:00+00:00  102.0548  4.129847
 2 2024-08-20 13:30:00+00:00  103.1252  4.068601,
 '912810ES3':                           Date     Price       YTM
 1040 2024-01-02 14:30:00+00:00  102.5495  4.454500
 1041 2024-01-03 14:30:00+00:00  1

# Fetching CT Yields:
- CT Yields: Market observed yields of on-the-runs (most recent issue for respective bill, note, bonds) 
- Default yields are EOD price markings according to FedInvest converted to YTM with rateslibs
- Able to enable bid, offer, mide side yields as of ~3:30 PM CST
- All tenors are returned by default, able to pass in specific tenor strings

In [6]:
ct_yields_df = cusip_curve_builder.fedinvest_data_fetcher.get_historical_ct_yields(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 8, 19),
)
ct_yields_df

,Date,CT1M,CT2M,CT3M,CT4M,CT6M,CT1,CT2,CT3,CT5,CT7,CT10,CT20,CT30
0,2024-01-02,5.305595,5.317435,5.295390,5.295062,5.177835,4.787577,4.370243,4.100865,3.928533,3.961870,3.944570,4.247731,4.083522
1,2024-01-03,5.290133,5.322151,5.304862,5.289391,5.187600,4.797913,4.332576,4.077939,3.895921,3.914869,3.906943,4.210063,4.053253
2,2024-01-04,5.289354,5.321362,5.293835,5.288615,5.186854,4.830198,4.382398,4.145585,3.972914,3.992895,3.989457,4.295486,4.137732
3,2024-01-05,5.287034,5.308855,5.307206,5.275949,5.180070,4.828254,4.399235,4.178916,4.015292,4.040103,4.049632,4.363723,4.208215
4,2024-01-08,5.276183,5.308078,5.306426,5.275178,5.168807,4.805679,4.365890,4.121773,3.966166,3.993101,4.011612,4.322542,4.177047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2024-08-13,5.255297,5.216028,5.131426,5.067974,4.873409,4.393958,3.939084,3.768494,3.679846,3.734021,3.847852,4.243769,4.162615
156,2024-08-14,5.056864,5.133108,5.099243,5.058926,4.917192,4.511028,4.088374,3.903772,3.790247,3.825416,3.919771,4.277996,4.181830
157,2024-08-15,5.470296,5.308716,5.202875,5.136830,4.921835,4.427609,3.954387,3.767335,3.672564,3.718496,3.828714,4.209615,4.122614
158,2024-08-16,5.256830,5.227680,5.143142,5.079722,4.916689,4.489286,4.069662,3.872618,3.768861,3.804479,3.890126,4.259539,4.152863


# Fetching CMT Yields:

In [7]:
cmt_yields_fred_df = cusip_curve_builder.fred_data_fetcher.get_historical_cmt_yields(
    start_date=datetime(2000, 1, 1),
    end_date=datetime(2024, 8, 19),
)
cmt_yields_fred_df

Fetching from FRED...


,Date,CMT3M,CMT6M,CMT1,CMT2,CMT3,CMT5,CMT7,CMT10,CMT20,CMT30
0,2000-01-03,5.27,5.55,6.09,6.38,6.42,6.50,6.65,6.58,6.94,6.61
1,2000-01-04,5.27,5.50,6.00,6.30,6.34,6.40,6.56,6.49,6.84,6.53
2,2000-01-05,5.28,5.49,6.05,6.38,6.43,6.51,6.68,6.62,6.95,6.64
3,2000-01-06,5.25,5.44,6.03,6.35,6.39,6.46,6.63,6.57,6.86,6.58
4,2000-01-07,5.22,5.42,6.00,6.31,6.35,6.42,6.58,6.52,6.82,6.55
...,...,...,...,...,...,...,...,...,...,...,...
6157,2024-08-13,5.06,4.75,4.40,3.93,3.75,3.68,3.74,3.85,4.25,4.16
6158,2024-08-14,5.07,4.77,4.42,3.94,3.76,3.67,3.72,3.83,4.22,4.12
6159,2024-08-15,5.09,4.82,4.52,4.08,3.90,3.79,3.83,3.92,4.28,4.18
6160,2024-08-16,5.08,4.80,4.49,4.06,3.87,3.77,3.81,3.89,4.26,4.15


# Pulling all Active CUSIPs given date (Bills, Notes, Bonds):
- Able to pull in respective CUSIP's auction results
- Latest (wrt paseed in as of date) STRIPping activity
- Latest (wrt pass in as of date) SOMA Holdings

In [8]:
curve_set_df = cusip_curve_builder.build_curve_set(
    as_of_date=datetime(2024, 9, 5),
    # include_auction_results=True,
    # include_soma_holdings=True,
    # include_stripping_activity=True,
    sorted=True, # recommended to enable this
    use_github=True, # recommended to enable this
)
curve_set_df

,cusip,security_type,auction_date,issue_date,maturity_date,time_to_maturity,int_rate,high_investment_rate,is_on_the_run,ust_label,security_term,original_security_term,offer_price,offer_yield,bid_price,bid_yield,mid_price,mid_yield,eod_price,eod_yield
0,912797LG0,Bill,2024-08-08,2024-08-13,2024-09-10,0.013699,NaN,5.381,False,5.381% Sep-24,4-Week,17-Week,0.000000,NaN,99.927361,6.542262,49.963681,NaN,99.942000,5.223029
1,912797LH8,Bill,2024-08-15,2024-08-20,2024-09-17,0.032877,NaN,5.355,False,5.355% Sep-24,4-Week,17-Week,99.826667,5.682566,99.826333,5.693535,99.826500,5.688051,99.841111,5.208279
2,912797LJ4,Bill,2024-08-22,2024-08-27,2024-09-24,0.052055,NaN,5.335,False,5.335% Sep-24,4-Week,17-Week,99.727931,5.456225,99.727667,5.461534,99.727799,5.458879,99.741500,5.183399
3,912797LK1,Bill,2024-08-29,2024-09-03,2024-10-01,0.071233,NaN,5.263,True,5.263% Oct-24,4-Week,17-Week,99.632750,5.307893,99.632389,5.313130,99.632570,5.310512,99.647917,5.087909
4,912797LS4,Bill,2024-08-08,2024-08-13,2024-10-08,0.090411,NaN,5.305,False,5.305% Oct-24,8-Week,17-Week,99.534333,5.263263,99.533417,5.273665,99.533875,5.268464,99.549333,5.092956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,912810TT5,Bond,2023-10-12,2023-10-16,2053-08-15,28.961644,4.125,NaN,False,4.125% Aug-53,29-Year 10-Month,30-Year,100.843750,4.074831,100.812500,4.076669,100.828125,4.075750,101.406250,4.041880
391,912810TV0,Bond,2024-01-11,2024-01-16,2053-11-15,29.213699,4.750,NaN,False,4.750% Nov-53,29-Year 10-Month,30-Year,111.781250,4.057024,111.750000,4.058722,111.765625,4.057873,112.343750,4.026584
392,912810TX6,Bond,2024-04-11,2024-04-15,2054-02-15,29.465753,4.250,NaN,False,4.250% Feb-54,29-Year 10-Month,30-Year,103.109375,4.067603,103.093750,4.068499,103.101562,4.068051,103.687500,4.034588
393,912810UA4,Bond,2024-07-11,2024-07-15,2054-05-15,29.709589,4.625,NaN,False,4.625% May-54,29-Year 10-Month,30-Year,109.734375,4.057682,109.718750,4.058535,109.726562,4.058109,110.343750,4.024576


# Pulling Historical Curve Sets:
- Able to fit a spline with a curve set filter function

In [9]:
def liquidity_premium_curve_set_filter(curve_set_df: pd.DataFrame):
    # remove OTR, olds, double olds, triple olds
    curve_set_filtered_df = curve_set_df[
        (curve_set_df["rank"] != 0) & (curve_set_df["rank"] != 1) & (curve_set_df["rank"] != 2) & (curve_set_df["rank"] != 3)
    ]

    # remove TBills
    curve_set_filtered_df = curve_set_filtered_df[curve_set_filtered_df["security_type"] != "Bill"]

    # remove low free float bonds (< $5bn)
    curve_set_filtered_df = curve_set_filtered_df[curve_set_filtered_df["free_float"] > 5000]

    # filter out bonds very close to maturity
    curve_set_filtered_df = curve_set_filtered_df[curve_set_filtered_df["time_to_maturity"] > 30 / 360]


    curve_set_filtered_df = curve_set_filtered_df.sort_values(by=["time_to_maturity"])

    return curve_set_filtered_df


In [10]:
start_date = datetime(2024, 9, 1)
end_date = datetime(2024, 9, 30)
quote_type = "eod"
curve_sets_dict_df, fitted_curves_dict = cusip_curve_builder.fetch_historical_curve_sets(
    start_date=start_date,
    end_date=end_date,
    fetch_soma_holdings=True,
    fetch_stripping_data=True,
    calc_free_float=True,
    fitted_curves=[
        ("LPF", f"{quote_type}_yield", liquidity_premium_curve_set_filter),
    ],
)

AGGREGATING CURVE SET DFs: 100%|██████████| 28/28 [00:00<00:00, 63.93it/s]


In [11]:
curve_sets_dict_df

{datetime.datetime(2024, 9, 3, 0, 0):          cusip security_type auction_date issue_date maturity_date  \
 0    9128286S4          Note   2019-04-25 2019-04-30    2026-04-30   
 1    912797KM8          Bill   2024-06-24 2024-06-27    2024-09-26   
 2    91282CFY2          Note   2022-11-22 2022-11-30    2029-11-30   
 3    912797ML8          Bill   2024-08-26 2024-08-29    2025-02-27   
 4    91282CDB4          Note   2021-10-12 2021-10-15    2024-10-15   
 ..         ...           ...          ...        ...           ...   
 389  91282CKS9          Note   2024-05-28 2024-05-31    2026-05-31   
 390  912810SK5          Bond   2020-01-09 2020-01-15    2049-11-15   
 391  91282CKZ3          Note   2024-07-09 2024-07-15    2027-07-15   
 392  91282CCZ2          Note   2021-09-27 2021-09-30    2026-09-30   
 393  9128284V9          Note   2018-10-10 2018-10-15    2028-08-15   
 
     price_per100 allocation_pctage avg_med_yield bid_to_cover_ratio  \
 0      99.673481         52.620000  

In [12]:
fitted_curves_dict

{datetime.datetime(2024, 9, 3, 0, 0): {'LPF': <CurvyCUSIPs.CurveInterpolator.GeneralCurveInterpolator at 0x2a315715e80>},
 datetime.datetime(2024, 9, 4, 0, 0): {'LPF': <CurvyCUSIPs.CurveInterpolator.GeneralCurveInterpolator at 0x2a30af66e90>},
 datetime.datetime(2024, 9, 5, 0, 0): {'LPF': <CurvyCUSIPs.CurveInterpolator.GeneralCurveInterpolator at 0x2a30af64910>},
 datetime.datetime(2024, 9, 6, 0, 0): {'LPF': <CurvyCUSIPs.CurveInterpolator.GeneralCurveInterpolator at 0x2a30aa70fc0>},
 datetime.datetime(2024, 9, 9, 0, 0): {'LPF': <CurvyCUSIPs.CurveInterpolator.GeneralCurveInterpolator at 0x2a30aa70c30>},
 datetime.datetime(2024, 9, 10, 0, 0): {'LPF': <CurvyCUSIPs.CurveInterpolator.GeneralCurveInterpolator at 0x2a30a86e570>},
 datetime.datetime(2024, 9, 11, 0, 0): {'LPF': <CurvyCUSIPs.CurveInterpolator.GeneralCurveInterpolator at 0x2a3156f6f10>},
 datetime.datetime(2024, 9, 12, 0, 0): {'LPF': <CurvyCUSIPs.CurveInterpolator.GeneralCurveInterpolator at 0x2a3156f7240>},
 datetime.datetime(20

# Pulling USD SOFR Swaps

In [18]:
def interp_func(x, y, x_new):
    f = scipy.interpolate.interp1d(x, y, kind="linear", fill_value="extrapolate", bounds_error=False)
    return f(x_new)


cusip_curve_builder.bbg_sef_fetcher.get_usd_sofr_swaps_historical_term_structures(
    start_date=datetime(2024, 1, 1), end_date=datetime(2024, 11, 1), liquid_tenors=True, interp_extrap_strat=interp_func
)

,Date,USD SWAP VS SOFR 1M,USD SWAP VS SOFR 2M,USD SWAP VS SOFR 3M,USD SWAP VS SOFR 4M,USD SWAP VS SOFR 6M,USD SWAP VS SOFR 9M,USD SWAP VS SOFR 1Y,USD SWAP VS SOFR 18M,USD SWAP VS SOFR 2Y,...,USD SWAP VS SOFR 11Y,USD SWAP VS SOFR 12Y,USD SWAP VS SOFR 15Y,USD SWAP VS SOFR 20Y,USD SWAP VS SOFR 25Y,USD SWAP VS SOFR 30Y,USD SWAP VS SOFR 35Y,USD SWAP VS SOFR 40Y,USD SWAP VS SOFR 45Y,USD SWAP VS SOFR 50Y
0,2024-01-02,5.34900,5.341535,5.33407,5.279267,5.16966,4.65641,4.80000,4.40587,4.14050,...,3.533995,3.54360,3.54960,3.52410,3.44250,3.34793,3.25336,3.15879,3.06422,2.96965
1,2024-01-03,5.34504,5.336045,5.32705,5.283013,5.19494,4.59060,4.82065,4.44705,4.15160,...,3.504150,3.52550,3.56450,3.52690,3.45950,3.37414,3.28878,3.20342,3.11806,3.03270
2,2024-01-04,5.33855,5.347000,4.20150,5.292300,3.72942,5.00735,4.86222,4.48613,4.22600,...,3.615500,3.61600,3.63686,3.62300,3.53630,3.45581,3.34310,3.23039,3.11768,3.00497
3,2024-01-05,5.33850,5.335500,5.33250,5.274817,5.15945,5.01715,4.84250,4.47915,4.21850,...,3.629900,3.65800,3.68691,3.67195,3.57460,3.50450,3.40009,3.29568,3.19127,3.08686
4,2024-01-08,5.33650,5.058780,5.06813,5.290800,4.41896,4.61850,4.84000,4.47000,4.21570,...,3.617000,3.65819,3.64273,3.67318,3.57755,3.49070,3.39340,3.29610,3.19880,3.10150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,2024-10-28,4.69850,4.630680,4.20500,4.277890,4.42367,4.28387,4.17338,3.99420,3.91013,...,3.795940,3.81835,3.85300,3.84111,3.80100,3.69115,3.61260,3.53405,3.45550,3.37695
214,2024-10-29,4.68150,4.633400,4.57850,4.519227,4.40068,4.25263,4.14221,3.97640,3.86910,...,3.838810,3.78971,3.86350,3.81131,3.74496,3.66441,3.63070,3.50723,3.38376,3.26029
215,2024-10-30,4.67260,4.623000,4.56850,4.511500,4.42500,4.29709,4.19530,4.01115,3.95244,...,3.761400,3.81150,3.83500,3.83776,3.75763,3.66669,3.57252,3.47835,3.38418,3.29001
216,2024-10-31,4.64948,4.604000,4.03640,4.507600,4.41100,4.28343,4.19086,4.00650,3.94560,...,3.786780,3.79446,3.79613,3.80132,3.74928,3.63948,3.55762,3.47576,3.39390,3.31204


# Pulling from WSJ/MarketWatch
- Need to know their ticker format

In [13]:
cusip_curve_builder.wsj_data_fetcher.wsj_timeseries_api(
    wsj_ticker_keys=[
        "FUTURE/US/XCBT/FFZ24",
        "FUTURE/US/XCBT/FFZ25",
        "FUTURE/US/XCME/SR3CZ24",
        "FUTURE/US/XCME/SR3CZ25",
    ]
)

{'FUTURE/US/XCBT/FFZ24':            Date  FUTURE/US/XCBT/FFZ24
 0    2020-09-28                99.980
 1    2020-09-29                99.980
 2    2020-09-30                99.980
 3    2020-10-01                99.980
 4    2020-10-02                99.980
 ...         ...                   ...
 1030 2024-11-01                95.505
 1031 2024-11-04                95.505
 1032 2024-11-05                95.495
 1033 2024-11-06                95.490
 1034 2024-11-08                95.500
 
 [1035 rows x 2 columns],
 'FUTURE/US/XCBT/FFZ25':           Date  FUTURE/US/XCBT/FFZ25
 0   2021-01-05                99.870
 1   2021-01-06                99.855
 2   2021-01-07                99.845
 3   2021-01-08                99.850
 4   2021-01-11                99.850
 ..         ...                   ...
 963 2024-11-01                96.350
 964 2024-11-04                96.365
 965 2024-11-05                96.305
 966 2024-11-06                96.220
 967 2024-11-08                96.280


# Pulling From Yahoo Finance

In [14]:
cusip_curve_builder.yf_data_fetcher.fetch_yf_legacy_multi_tickers(
    tickers=["TLT", "EDV", "ZROZ"],
    from_date=datetime(2024, 1, 1),
    to_date=datetime(2024, 9, 1),
)

{'TLT':            Date       Open       High        Low      Close  Adj Close  \
 0    2024-01-02  98.230003  98.650002  98.000000  98.309998  95.434738   
 1    2024-01-03  97.379997  98.849998  97.150002  98.720001  95.832748   
 2    2024-01-04  97.419998  97.639999  97.070000  97.220001  94.376617   
 3    2024-01-05  96.500000  97.680000  96.209999  96.290001  93.473808   
 4    2024-01-08  96.230003  97.379997  96.089996  97.239998  94.396027   
 ..          ...        ...        ...        ...        ...        ...   
 163  2024-08-26  98.690002  98.699997  98.080002  98.139999  97.492577   
 164  2024-08-27  97.489998  98.080002  97.410004  97.970001  97.323700   
 165  2024-08-28  98.010002  98.199997  97.730003  97.849998  97.204491   
 166  2024-08-29  97.370003  97.639999  97.129997  97.529999  96.886604   
 167  2024-08-30  97.680000  97.989998  96.470001  96.489998  95.853455   
 
        Volume  
 0    47797400  
 1    58159500  
 2    52604500  
 3    45883000  
 4    

In [15]:
cusip_curve_builder.yf_data_fetcher.fetch_yf_legacy(
    ticker="TLT",
    from_date=datetime(2024, 1, 1),
    to_date=datetime(2024, 9, 1),
)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2024-01-02,98.230003,98.650002,98.000000,98.309998,95.434731,47797400
1,2024-01-03,97.379997,98.849998,97.150002,98.720001,95.832756,58159500
2,2024-01-04,97.419998,97.639999,97.070000,97.220001,94.376602,52604500
3,2024-01-05,96.500000,97.680000,96.209999,96.290001,93.473816,45883000
4,2024-01-08,96.230003,97.379997,96.089996,97.239998,94.396027,39485800
...,...,...,...,...,...,...,...
163,2024-08-26,98.690002,98.699997,98.080002,98.139999,97.492577,23198700
164,2024-08-27,97.489998,98.080002,97.410004,97.970001,97.323700,21507700
165,2024-08-28,98.010002,98.199997,97.730003,97.849998,97.204491,18501400
166,2024-08-29,97.370003,97.639999,97.129997,97.529999,96.886597,24988200


# Pulling From FRED

In [16]:
cusip_curve_builder.fred_data_fetcher.fred.get_multiple_series(series_ids=["M2SL", "M2REAL"], one_df=True)

,M2SL,M2REAL
1959-01-01,286.6,987.9
1959-02-01,287.7,992.1
1959-03-01,289.2,998.3
1959-04-01,290.1,1001.0
1959-05-01,292.2,1006.2
...,...,...
2024-05-01,20959.4,6691.5
2024-06-01,21020.1,6714.6
2024-07-01,21039.4,6710.4
2024-08-01,21141.2,6730.3
